In [1]:
import pickle

In [ ]:
with open
pickle.load()

In [ ]:
#generate a new df containing the classes corresponding to Butchniks emotions
classes = anp_df.emotion_label.unique()
from sklearn.preprocessing import label_binarize
anp2 = pd.DataFrame(label_binarize(anp_df.emotion_label, classes=classes),columns=classes)
#put the value of emotion score in the corresponding row of the image
anp2 = anp2.multiply(anp_df.emotion_score,axis=0)

In [ ]:
anp2.head()

In [ ]:
#concatenate anp2 with image id and ANP sentiment
anp_final = pd.concat([anp_df[['image_id','anp_sentiment']], anp2], axis=1)
anp_final.head()